<a href="https://colab.research.google.com/github/yjyjy131/Study_Deep_Learning/blob/main/Deep-learning-from-scratch/chapter_7_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 4차원 배열 이미지를 2차원 배열로 변환

In [1]:
import sys, os
import numpy as np

In [62]:
# 4차원 배열 이미지 (수, 채널, 높이, 너비) 다수를 입력받아 2차원 배열로 변환
def img2col(input_data, filter_h, filter_w, stride=1, pad=0) :
  N, C, H, W = input_data.shape

  # 필터 적용시 h, w 의 크기 계산 수식
  out_h = (H + 2*pad - filter_h) // stride + 1
  out_w = (W + 2*pad - filter_w) // stride + 1

  # input_data 에서 높이, 너비에만 padding 값을 더함
  img = np.pad(input_data, [(0, 0), (0, 0), (pad, pad), (pad, pad)], 'constant')
  col = np.zeros((N, C, filter_h, filter_w, out_h, out_w))

  for y in range(filter_h) :
    y_max = y + stride * out_h
    for x in range(filter_w) :
      x_max = x + stride * out_w
      # col[...] 의 자리에 차례대로 img[...] 의 값을 넣는다
      col[:, :, y, x, :, :] = img[:, :, y:y_max:stride, x:x_max:stride]

  # 원하는 column 원소 순서대로 전치 
  col = col.transpose(0, 4, 5, 1, 2, 3).reshape(N * out_h * out_w -1)

  return col

### Convolution layer 구현하기

In [61]:
class Convolution:
  def __init__ (self, W, b, stride=1, pad=0) :
    self.W = W
    self.b = b
    self.stride = stride
    self.pad = pad

  def forward(self, x) :
    # 필터는 CNN 의 가중치에 해당한다
    FN, C, FH, FW = self.W.shape
    N, C, H, W = x.shape
    out_h = int( 1 + (H + 2*self.pad - FH) / self.stride)
    out_w = int( 1 + (W + 2*self.pad - FW) / self.stride)

    # 입력데이터, 필터 전개 
    col = img2col(x, FH, FW, self.stride, self.pad)
    # -1 을 함께 입력해 reshape 해야 원소 수를 똑같이 유지하며 변형 가능
    # -1 을 사용하지 않을 시 reshape error 발생
    # FN * C * FH * FW 개 데이터를 (FN, C*CH*FW) 의 데이터 shape 으로 변환
    col_W = self.W.reshape(FN, -1).T

    # 필터 연산
    out = np.dot(col, col_W) + self.b

    out = out.reshape(N, out_h, out_w, -1).tranpose(0, 3, 1, 2)

    return out